In [5]:
import pandas as pd
import nltk
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, classification_report
from imblearn.over_sampling import RandomOverSampler

# Carregar os dados
data = pd.read_csv('new_comentarios_ingles.csv')
df_1 = data['Comentario']

df['Comentario'] = df['Comentario'].apply(lambda x: x.lower())

def define_sentiment(rating):
    if rating == 'positivo':
        return 1
    elif rating == 'neutro':
        return 2
    else:
        return 0
    
data['sentimento_polarizado'] = data['sentimento'].apply(define_sentiment)

rotulos = data['sentimento_polarizado']

# Tokenização e lematização
comentarios_tokenizados = []

for tok in df_1:
    tokens = nltk.word_tokenize(str(tok))
    comentarios_tokenizados.append(tokens)

# Divisão em conjuntos de treinamento e teste
X_treino, X_teste, y_treino, y_teste = train_test_split(comentarios_tokenizados, rotulos, test_size=0.2, random_state=42, stratify=rotulos)

# Vetorização dos dados de texto
vectorizer = CountVectorizer()
X_treino_vect = vectorizer.fit_transform([' '.join(comentario) for comentario in X_treino])
X_teste_vect = vectorizer.transform([' '.join(comentario) for comentario in X_teste])

# Aplicando Random Oversampler
oversampler = RandomOverSampler()
X_treino_res, y_treino_res = oversampler.fit_resample(X_treino_vect, y_treino)

# Treinamento do classificador Naive Bayes
naive_bayes_classifier = MultinomialNB()

# Validação cruzada
scores = cross_val_score(naive_bayes_classifier, X_treino_res, y_treino_res, cv=5)

# Treinamento final do modelo Naive Bayes
naive_bayes_classifier.fit(X_treino_res, y_treino_res)

# Predição nos dados de teste usando Naive Bayes
predicoes = naive_bayes_classifier.predict(X_teste_vect)

# Calcular precisão, recall e F-score para Naive Bayes
precision = precision_score(y_teste, predicoes, average='weighted')
recall = recall_score(y_teste, predicoes, average='weighted')
f_score = f1_score(y_teste, predicoes, average='weighted')
acuracia = accuracy_score(y_teste, predicoes)
report = classification_report(y_teste, predicoes, target_names=['Negativo', 'Positivo', 'Neutro'])

# Imprimir resultados
print("Resultados da validação cruzada:")
print("Scores:", scores)
print("Precisão média:", scores.mean())
print("Acuracia: {:.2f}".format(acuracia))
print("Precisão: {:.2f}".format(precision))
print("Recall: {:.2f}".format(recall))
print("F-score: {:.2f}".format(f_score))
print("\nClassification Report:")
print(report)


Resultados da validação cruzada:
Scores: [0.81769505 0.82187938 0.81448425 0.82047686 0.81818182]
Precisão média: 0.818543473286917
Acuracia: 0.69
Precisão: 0.82
Recall: 0.69
F-score: 0.74

Classification Report:
              precision    recall  f1-score   support

    Negativo       0.17      0.41      0.24       220
    Positivo       0.92      0.74      0.82      3268
      Neutro       0.17      0.37      0.24       317

    accuracy                           0.69      3805
   macro avg       0.42      0.51      0.43      3805
weighted avg       0.82      0.69      0.74      3805

